In [ ]:
# Start your code here!
import pandas as pd
import matplotlib.pyplot as plt
import pingouin
import scipy.stats as scp

men=pd.read_csv("men_results.csv")
women=pd.read_csv("women_results.csv")

men["date"]=pd.to_datetime(men["date"])
women["date"]=pd.to_datetime(women["date"])

men_subset=men.query('(date > "2002-01-01") & (tournament in ("FIFA World Cup"))')
women_subset=women.query('(date > "2002-01-01") & (tournament in ("FIFA World Cup"))')

men_subset['group']='men'
women_subset['group']='women'

men_subset['goals_scored']=men_subset["home_score"] + men_subset["away_score"]
women_subset['goals_scored']=women_subset["home_score"] + women_subset["away_score"]

men_subset['goals_scored'].hist()
plt.title('Men Goals Scored')
plt.show()
plt.clf()
women_subset['goals_scored'].hist()
plt.title('Women Goals Scored')
plt.show()
plt.clf()

both=pd.concat([men_subset,women_subset], axis=0, ignore_index=True)
both_subset=both[["group","goals_scored"]]
both_subset_wide=pd.pivot(data=both_subset,columns="group", values="goals_scored")

# Perform right-tailed Wilcoxon-Mann-Whitney test with pingouin
results_pg = pingouin.mwu(x=both_subset_wide["women"],
                          y=both_subset_wide["men"],
                          alternative="greater")

# Alternative SciPy solution: Perform right-tailed Wilcoxon-Mann-Whitney test with scipy
results_scipy = scp.mannwhitneyu(x=women_subset["goals_scored"],
                             y=men_subset["goals_scored"],
                             alternative="greater")

p_val=results_pg['p-val'].values[0]

if p_val <= 0.1 :
    result = 'reject'
else:
    result= 'fail to reject'
    
result_dict = {"p_val": p_val, "result": result}